In [ ]:
pip install scx>=1.0.7

In [ ]:
from scx.optimize import Model

#QUESTION 1 - MINIMIZE COST

In [ ]:
transport = [
    {
        'origin name': 'A1',
        'destination name': 'R1',
        'distance': 105,
        'cost_per_mile': 0.12
    },
    {
        'origin name': 'A1',
        'destination name': 'R2',
        'distance': 256,
        'cost_per_mile': 0.12
     },
    {
        'origin name': 'A1',
        'destination name': 'R3',
        'distance': 86,
        'cost_per_mile': 0.12
    },
    {
        'origin name': 'A2',
        'destination name': 'R1',
        'distance': 240,
        'cost_per_mile': 0.12
    },
    {
        'origin name': 'A2',
        'destination name': 'R2',
        'distance': 136,
        'cost_per_mile': 0.12
    },
    {
        'origin name': 'A2',
        'destination name': 'R3',
        'distance': 198,
        'cost_per_mile': 0.12
    },
]

In [ ]:
demand = [{
        'name':'R1',
        'demand': 2500
    },
    {
        'name':'R2',
        'demand': 4350
    },
    {
        'name':'R3',
        'demand': 3296
}]

In [ ]:
for t in transport:
  #create decision variables for each item in transport
    t['amt'] = Model.variable(name=f"{t['origin name']}_{t['destination name']}_amt", lowBound=0)
  #calculate the variable cost of shipping for each item in transport
    t['cost'] = t['distance'] * t['cost_per_mile']

In [ ]:
print(transport)

[{'origin name': 'A1', 'destination name': 'Region 1', 'distance': 105, 'cost_per_mile': 0.12, 'amt': A1_Region_1_amt, 'cost': 12.6}, {'origin name': 'A1', 'destination name': 'Region 2', 'distance': 256, 'cost_per_mile': 0.12, 'amt': A1_Region_2_amt, 'cost': 30.72}, {'origin name': 'A1', 'destination name': 'Region 3', 'distance': 86, 'cost_per_mile': 0.12, 'amt': A1_Region_3_amt, 'cost': 10.32}, {'origin name': 'A2', 'destination name': 'Region 1', 'distance': 240, 'cost_per_mile': 0.12, 'amt': A2_Region_1_amt, 'cost': 28.799999999999997}, {'origin name': 'A2', 'destination name': 'Region 2', 'distance': 136, 'cost_per_mile': 0.12, 'amt': A2_Region_2_amt, 'cost': 16.32}, {'origin name': 'A2', 'destination name': 'Region 3', 'distance': 198, 'cost_per_mile': 0.12, 'amt': A2_Region_3_amt, 'cost': 23.759999999999998}]


In [ ]:
for t in transport:
    print(f"From {t['origin name']} to {t['destination name']}: {t['cost']:.2f}")

From A1 to R1: 12.60
From A1 to R2: 30.72
From A1 to R3: 10.32
From A2 to R1: 28.80
From A2 to R2: 16.32
From A2 to R3: 23.76


In [ ]:
#Initialize my model
my_model=Model(name="Blink22", sense="minimize")

#add the objective function
my_model.add_objective(
    fn=Model.sum([t['cost'] * t['amt'] for t in transport])
)

#add the constraint
##demand constraint
for d in demand:
    my_model.add_constraint(
        name=f"{d['name']}_demand",
        fn=Model.sum([t['amt'] for t in transport if t['destination name']==d['name']])>=d['demand']
        )

#solve model
my_model.solve()

In [ ]:
my_model.show_formulation()

Blink22:
MINIMIZE
12.6*A1_R1_amt + 30.72*A1_R2_amt + 10.32*A1_R3_amt + 28.799999999999997*A2_R1_amt + 16.32*A2_R2_amt + 23.759999999999998*A2_R3_amt + 0.0
SUBJECT TO
R1_demand: A1_R1_amt + A2_R1_amt >= 2500

R2_demand: A1_R2_amt + A2_R2_amt >= 4350

R3_demand: A1_R3_amt + A2_R3_amt >= 3296

VARIABLES
A1_R1_amt Continuous
A1_R2_amt Continuous
A1_R3_amt Continuous
A2_R1_amt Continuous
A2_R2_amt Continuous
A2_R3_amt Continuous



In [ ]:
#show the outputs
my_model.show_outputs()

{'objective': 136506.72,
 'status': 'Optimal',
 'variables': {'A1_R1_amt': 2500.0,
               'A1_R2_amt': 0.0,
               'A1_R3_amt': 3296.0,
               'A2_R1_amt': 0.0,
               'A2_R2_amt': 4350.0,
               'A2_R3_amt': 0.0}}


#QUESTION 2 - MAXIMIZE PROFIT

In [ ]:
revenue = 45

In [ ]:
for t in transport:
  #create decision variables for each item in transport
    t['amt'] = Model.variable(name=f"{t['origin name']}_{t['destination name']}_amt", lowBound=0)
  #calculate the variable cost of shipping for each item in transport
    t['cost'] = t['distance'] * t['cost_per_mile']

    #poderia ter calculado o lucro de uma vez:
    #t['profit'] = 45-t['cost']
    #depois inves de calcular o revenue de cada regiao - o custo, poderia ter calculado diretamente a soma do profit

In [ ]:
for d in demand:
    #calculate revenue
    d['revenue'] = [d['demand'] * revenue for d in demand]

In [ ]:
#Initialize my model
my_model=Model(name="Blink22.2", sense="maximize")

#new objective function
my_model.add_objective(
    fn=Model.sum(d['revenue'])-Model.sum([t['cost'] * t['amt'] for t in transport])
)
    #aqui ficaria apenas fn=Model.sum(t['profit']* t['amt'] for t in transport])

#add the constraint
##demand constraint
for d in demand:
    my_model.add_constraint(
        name=f"{d['name']}_demand",
        fn=Model.sum([t['amt'] for t in transport if t['destination name']==d['name']])>=d['demand']
        )

#solve model
my_model.solve()

In [ ]:
my_model.show_outputs()

{'objective': 320063.28,
 'status': 'Optimal',
 'variables': {'A1_R1_amt': 2500.0,
               'A1_R2_amt': 0.0,
               'A1_R3_amt': 3296.0,
               'A2_R1_amt': 0.0,
               'A2_R2_amt': 4350.0,
               'A2_R3_amt': 0.0}}


#QUESTION 3 - SUPPLY CONSTRAINT

In [ ]:
capacity = [
    {
        'name':'A1',
        'capacity': 13000
        },
    {
        'name':'A2',
        'capacity': 3500
    }
]

In [ ]:
for t in transport:
  #create decision variables for each item in transport
    t['amt'] = Model.variable(name=f"{t['origin name']}_{t['destination name']}_amt", lowBound=0)
  #calculate the variable cost of shipping for each item in transport
    t['cost'] = t['distance'] * t['cost_per_mile']

In [ ]:
#Initialize my model
my_model=Model(name="Blink22.3", sense="minimize")

#add the objective function
my_model.add_objective(
    fn=Model.sum([t['cost'] * t['amt'] for t in transport])
)

#add the constraint
##demand constraint
for d in demand:
    my_model.add_constraint(
        name=f"{d['name']}_demand",
        fn=Model.sum([t['amt'] for t in transport if t['destination name']==d['name']])==d['demand']
        )
##supply constraint
for c in capacity:
    my_model.add_constraint(
        name=f"{c['name']}_supply",
        fn=Model.sum([t['amt'] for t in transport if t['origin name']==c['name']])<=c['capacity']
    )

#solve model
my_model.solve()

In [ ]:
my_model.show_outputs()

{'objective': 148746.72,
 'status': 'Optimal',
 'variables': {'A1_R1_amt': 2500.0,
               'A1_R2_amt': 850.0,
               'A1_R3_amt': 3296.0,
               'A2_R1_amt': 0.0,
               'A2_R2_amt': 3500.0,
               'A2_R3_amt': 0.0}}


#QUESTION 4 - DISTRIBUTION CENTER (CONSERVATION OF FLOW)

In [ ]:
transport_dc = [
    {
        'origin name': 'A1',
        'destination name': 'DC',
        'distance': 190,
        'cost_per_mile': 0.04
    },
    {
        'origin name': 'A2',
        'destination name': 'DC',
        'distance': 150,
        'cost_per_mile': 0.04
     },
    {
        'origin name': 'DC',
        'destination name': 'R1',
        'distance': 15,
        'cost_per_mile': 0.08
    },
    {
        'origin name': 'DC',
        'destination name': 'R2',
        'distance': 59,
        'cost_per_mile': 0.08
    },
    {
        'origin name': 'DC',
        'destination name': 'R3',
        'distance': 79,
        'cost_per_mile': 0.08
    }
]

In [ ]:
distribution_center = [
    {
        'name':'DC',
    }
]

In [ ]:
for x in transport_dc:
  #create decision variables for each item in transport
    x['amt'] = Model.variable(name=f"{x['origin name']}_{x['destination name']}_amt", lowBound=0)
  #calculate the variable cost of shipping for each item in transport
    x['cost'] = x['distance'] * x['cost_per_mile']

In [ ]:
for x in transport_dc:
    print(f"From {t['origin name']} to {t['destination name']}: {t['cost']:.2f}")

From A2 to R3: 23.76
From A2 to R3: 23.76
From A2 to R3: 23.76
From A2 to R3: 23.76
From A2 to R3: 23.76


In [ ]:
#Initialize my model
my_model=Model(name="Blink22.4", sense="minimize")

#add the objective function
my_model.add_objective(
    fn=Model.sum([x['cost']* x['amt'] for x in transport_dc])
)

#add the constraint
##demand constraint
for d in demand:
    my_model.add_constraint(
        name=f"{d['name']}_demand",
        fn=Model.sum([x['amt'] for x in transport_dc if x['destination name']==d['name']])>=d['demand']
        )

##supply constraint
for c in capacity:
    my_model.add_constraint(
        name=f"{c['name']}_supply",
        fn=Model.sum([x['amt'] for x in transport_dc if x['origin name']==c['name']])<=c['capacity']
    )

##conservation_of_flow constraint
for dc in distribution_center:
    my_model.add_constraint(
        name=f"{dc['name']}_flow",
        fn=Model.sum(
          [x['amt'] for x in transport_dc if x['destination name']==dc['name']]
        ) == Model.sum(
          [x['amt'] for x in transport_dc if x['origin name']==dc['name']]
        )
    )


#solve model
my_model.solve()

In [ ]:
my_model.show_outputs()


{'objective': 115872.32,
 'status': 'Optimal',
 'variables': {'A1_DC_amt': 6646.0,
               'A2_DC_amt': 3500.0,
               'DC_R1_amt': 2500.0,
               'DC_R2_amt': 4350.0,
               'DC_R3_amt': 3296.0}}


#QUESTION 5 - NEW DISTRIBUTION CENTER (BINARY)

In [ ]:
transport_dc2 = [
    {
        'origin name': 'A1',
        'destination name': 'DC1',
        'distance': 190,
        'cost_per_mile': 0.04
    },
    {
        'origin name': 'A1',
        'destination name': 'DC2',
        'distance': 15,
        'cost_per_mile': 0.04
    },
    {
        'origin name': 'A2',
        'destination name': 'DC1',
        'distance': 150,
        'cost_per_mile': 0.04
     },
    {
        'origin name': 'A2',
        'destination name': 'DC2',
        'distance': 36,
        'cost_per_mile': 0.04
    },
    {
        'origin name': 'DC1',
        'destination name': 'R1',
        'distance': 15,
        'cost_per_mile': 0.08
    },
    {
        'origin name': 'DC2',
        'destination name': 'R1',
        'distance': 135,
        'cost_per_mile': 0.08
    },
    {
        'origin name': 'DC1',
        'destination name': 'R2',
        'distance': 59,
        'cost_per_mile': 0.08
    },
    {
        'origin name': 'DC2',
        'destination name': 'R2',
        'distance': 45,
        'cost_per_mile': 0.08
    },
    {
        'origin name': 'DC1',
        'destination name': 'R3',
        'distance': 79,
        'cost_per_mile': 0.08
    },
    {
        'origin name': 'DC2',
        'destination name': 'R3',
        'distance': 129,
        'cost_per_mile': 0.08
    },
]

In [ ]:
distribution_center_2 = [
    {
        'name':'DC1',
        'fixed_cost': 11500
    },
    {
        'name':'DC2',
        'fixed_cost': 15500
    }
]

In [ ]:
for x in transport_dc2:
  #create decision variables for each item in transport
    x['amt'] = Model.variable(name=f"{x['origin name']}_{x['destination name']}_amt", lowBound=0)
  #calculate the variable cost of shipping for each item in transport
    x['cost'] = x['distance'] * x['cost_per_mile']

In [ ]:
for dc in distribution_center_2:
  #create a usage variable
    dc['usage'] = Model.variable(name=f"{dc['name']}_usage", cat="Binary")

In [ ]:
#Initialize my model
my_model=Model(name="Blink22.5", sense="minimize")

#add the objective function
my_model.add_objective(
    fn=Model.sum(
        [x['cost']* x['amt'] for x in transport_dc2]
      )+Model.sum(
        [dc['fixed_cost']*dc['usage'] for dc in distribution_center_2])
)

#add the constraint
##demand constraint
for d in demand:
    my_model.add_constraint(
        name=f"{d['name']}_demand",
        fn=Model.sum([x['amt'] for x in transport_dc2 if x['destination name']==d['name']])>=d['demand']
        )

##supply constraint
for c in capacity:
    my_model.add_constraint(
        name=f"{c['name']}_supply",
        fn=Model.sum([x['amt'] for x in transport_dc2 if x['origin name']==c['name']])<=c['capacity']
    )

##conservation_of_flow constraint
for dc in distribution_center_2:
    my_model.add_constraint(
        name=f"{dc['name']}_flow",
        fn=Model.sum(
          [x['amt'] for x in transport_dc2 if x['destination name']==dc['name']]
        ) == Model.sum(
          [x['amt'] for x in transport_dc2 if x['origin name']==dc['name']]
        )
    )

##binary linking constraint
for dc in distribution_center_2:
    my_model.add_constraint(
        name=f"{dc['name']}_binary",
        fn=dc['usage'] == Model.sum(
          [x['amt'] for x in transport_dc2 if x['destination name']==dc['name']]
        ) <= 999999*dc['usage']
    )

#solve model
my_model.solve()

In [ ]:
my_model.show_formulation()

Blink22.5:
MINIMIZE
7.6000000000000005*A1_DC1_amt + 0.6*A1_DC2_amt + 6.0*A2_DC1_amt + 1.44*A2_DC2_amt + 1.2*DC1_R1_amt + 4.72*DC1_R2_amt + 6.32*DC1_R3_amt + 11500*DC1_usage + 10.8*DC2_R1_amt + 3.6*DC2_R2_amt + 10.32*DC2_R3_amt + 15500*DC2_usage + 0.0
SUBJECT TO
R1_demand: DC1_R1_amt + DC2_R1_amt >= 2500

R2_demand: DC1_R2_amt + DC2_R2_amt >= 4350

R3_demand: DC1_R3_amt + DC2_R3_amt >= 3296

A1_supply: A1_DC1_amt + A1_DC2_amt <= 13000

A2_supply: A2_DC1_amt + A2_DC2_amt <= 3500

DC1_flow: A1_DC1_amt + A2_DC1_amt - DC1_R1_amt - DC1_R2_amt - DC1_R3_amt = 0

DC2_flow: A1_DC2_amt + A2_DC2_amt - DC2_R1_amt - DC2_R2_amt - DC2_R3_amt = 0

DC1_binary: A1_DC1_amt + A2_DC1_amt - 999999 DC1_usage <= 0

DC2_binary: A1_DC2_amt + A2_DC2_amt - 999999 DC2_usage <= 0

VARIABLES
A1_DC1_amt Continuous
A1_DC2_amt Continuous
A2_DC1_amt Continuous
A2_DC2_amt Continuous
DC1_R1_amt Continuous
DC1_R2_amt Continuous
DC1_R3_amt Continuous
0 <= DC1_usage <= 1 Integer
DC2_R1_amt Continuous
DC2_R2_amt Continuous
DC2

In [ ]:
my_model.show_outputs()

{'objective': 98262.32,
 'status': 'Optimal',
 'variables': {'A1_DC1_amt': 0.0,
               'A1_DC2_amt': 10146.0,
               'A2_DC1_amt': 0.0,
               'A2_DC2_amt': 0.0,
               'DC1_R1_amt': 0.0,
               'DC1_R2_amt': 0.0,
               'DC1_R3_amt': 0.0,
               'DC1_usage': 0.0,
               'DC2_R1_amt': 2500.0,
               'DC2_R2_amt': 4350.0,
               'DC2_R3_amt': 3296.0,
               'DC2_usage': 1.0}}
